<a href="https://colab.research.google.com/github/pritul2/Detection-of-Person-With-or-Without-Mask/blob/master/Face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install onnx==1.6.0
!pip install onnx-tf==1.3.0
!pip install onnxruntime==0.5.0
!pip install opencv-python==4.1.1.26
!pip install tensorflow==1.13.1

     |████████████████████████████████| 4.8MB 4.8MB/s 
     |████████████████████████████████| 122kB 4.8MB/s 
     |████████████████████████████████| 3.2MB 4.6MB/s 
     |████████████████████████████████| 28.7MB 148kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


     |████████████████████████████████| 92.5MB 101kB/s 
     |████████████████████████████████| 368kB 51.9MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


In [0]:
!unzip /content/dataset_mask\ 2.zip

Archive:  /content/dataset_mask 2.zip
   creating: dataset_mask/
   creating: dataset_mask/mask images/
  inflating: dataset_mask/mask images/download (5).jpeg  
   creating: __MACOSX/
   creating: __MACOSX/dataset_mask/
   creating: __MACOSX/dataset_mask/mask images/
  inflating: __MACOSX/dataset_mask/mask images/._download (5).jpeg  
  inflating: dataset_mask/mask images/images (30).jpeg  
  inflating: __MACOSX/dataset_mask/mask images/._images (30).jpeg  
  inflating: dataset_mask/mask images/images (2).jpeg  
  inflating: __MACOSX/dataset_mask/mask images/._images (2).jpeg  
  inflating: dataset_mask/mask images/download.jpeg  
  inflating: __MACOSX/dataset_mask/mask images/._download.jpeg  
  inflating: dataset_mask/mask images/images (29).jpeg  
  inflating: __MACOSX/dataset_mask/mask images/._images (29).jpeg  
  inflating: dataset_mask/mask images/images (14).jpeg  
  inflating: __MACOSX/dataset_mask/mask images/._images (14).jpeg  
  inflating: dataset_mask/mask images/images 

In [0]:
import cv2
import onnx
import onnxruntime as ort
from onnx_tf.backend import prepare
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
def area_of(left_top, right_bottom):
    """
    Compute the areas of rectangles given two corners.
    Args:
        left_top (N, 2): left top corner.
        right_bottom (N, 2): right bottom corner.
    Returns:
        area (N): return the area.
    """
    hw = np.clip(right_bottom - left_top, 0.0, None)
    return hw[..., 0] * hw[..., 1]

def iou_of(boxes0, boxes1, eps=1e-5):
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Args:
        boxes0 (N, 4): ground truth boxes.
        boxes1 (N or 1, 4): predicted boxes.
        eps: a small number to avoid 0 as denominator.
    Returns:
        iou (N): IoU values.
    """
    overlap_left_top = np.maximum(boxes0[..., :2], boxes1[..., :2])
    overlap_right_bottom = np.minimum(boxes0[..., 2:], boxes1[..., 2:])

    overlap_area = area_of(overlap_left_top, overlap_right_bottom)
    area0 = area_of(boxes0[..., :2], boxes0[..., 2:])
    area1 = area_of(boxes1[..., :2], boxes1[..., 2:])
    return overlap_area / (area0 + area1 - overlap_area + eps)

def hard_nms(box_scores, iou_threshold, top_k=-1, candidate_size=200):
    """
    Perform hard non-maximum-supression to filter out boxes with iou greater
    than threshold
    Args:
        box_scores (N, 5): boxes in corner-form and probabilities.
        iou_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
        candidate_size: only consider the candidates with the highest scores.
    Returns:
        picked: a list of indexes of the kept boxes
    """
    scores = box_scores[:, -1]
    boxes = box_scores[:, :-1]
    picked = []
    indexes = np.argsort(scores)
    indexes = indexes[-candidate_size:]
    while len(indexes) > 0:
        current = indexes[-1]
        picked.append(current)
        if 0 < top_k == len(picked) or len(indexes) == 1:
            break
        current_box = boxes[current, :]
        indexes = indexes[:-1]
        rest_boxes = boxes[indexes, :]
        iou = iou_of(
            rest_boxes,
            np.expand_dims(current_box, axis=0),
        )
        indexes = indexes[iou <= iou_threshold]

    return box_scores[picked, :]

def predict(width, height, confidences, boxes, prob_threshold, iou_threshold=0.5, top_k=-1):
    """
    Select boxes that contain human faces
    Args:
        width: original image width
        height: original image height
        confidences (N, 2): confidence array
        boxes (N, 4): boxes array in corner-form
        iou_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
    Returns:
        boxes (k, 4): an array of boxes kept
        labels (k): an array of labels for each boxes kept
        probs (k): an array of probabilities for each boxes being in corresponding labels
    """
    boxes = boxes[0]
    confidences = confidences[0]
    picked_box_probs = []
    picked_labels = []
    for class_index in range(1, confidences.shape[1]):
        probs = confidences[:, class_index]
        mask = probs > prob_threshold
        probs = probs[mask]
        if probs.shape[0] == 0:
            continue
        subset_boxes = boxes[mask, :]
        box_probs = np.concatenate([subset_boxes, probs.reshape(-1, 1)], axis=1)
        box_probs = hard_nms(box_probs,
           iou_threshold=iou_threshold,
           top_k=top_k,
           )
        picked_box_probs.append(box_probs)
        picked_labels.extend([class_index] * box_probs.shape[0])
    if not picked_box_probs:
        return np.array([]), np.array([]), np.array([])
    picked_box_probs = np.concatenate(picked_box_probs)
    picked_box_probs[:, 0] *= width
    picked_box_probs[:, 1] *= height
    picked_box_probs[:, 2] *= width
    picked_box_probs[:, 3] *= height
    return picked_box_probs[:, :4].astype(np.int32), np.array(picked_labels), picked_box_probs[:, 4]

In [0]:
cnt = 1
def detect_face(image):
  global cnt
  #image = cv2.imread("/content/without_mask2/43400a0296ce70883f285009111c8524.jpg") #Changes done here#
  image = cv2.resize(image,(640,480))
  #plt.imshow(image)
  #plt.show()
  img = image.copy()
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_mean = np.array([127, 127, 127])
  img = (img - img_mean) / 128
  img = np.transpose(img, [2, 0, 1])
  img = np.expand_dims(img, axis=0)
  img = img.astype(np.float32)
  onnx_model = onnx.load('/content/drive/My Drive/Copy of ultra_light_640.onnx')
  predictor = prepare(onnx_model)
  ort_session = ort.InferenceSession('/content/drive/My Drive/Copy of ultra_light_640.onnx')
  input_name = ort_session.get_inputs()[0].name
  confidences, boxes = ort_session.run(None, {input_name: img})
  h,w,_ = image.shape
  #print(boxes)
  boxes, labels, probs = predict(w, h, confidences, boxes, 0.7)
  print(boxes, labels, probs)
  #input()
  flag=False
  for i in range(boxes.shape[0]):
        flag=True
        box = boxes[i, :]
        x1, y1, x2, y2 = box
        print(x1, y1, x2, y2)
        #cv2.rectangle(image, (x1, y1-5), (x2, y2+5), (80,18,236), 2)
        #cv2.rectangle(image, (x1, y2), (x2, y2), (80,18,236), 2)
        #print(image,type(image),image.shape,)
        #input()
        image = image[y1:y2,x1:x2]
        if image.shape[0]<=0 or image.shape[1]<=0 or len(image)==0:
          continue
        image = cv2.resize(image,(256,256))
        print(image.shape)
        path = "/content/withmask"+str(cnt)+".png"
        cv2.imwrite(path,image)
        cnt+=1
'''        plt.imshow(image)
        plt.show()'''
        
for i in os.listdir('/content/dataset_mask/mask images'):
  img = cv2.imread("/content/dataset_mask/mask images/"+i)
  #plt.imshow(img)
  #plt.show()
  print(i)
  detect_face(img)
  #input("done")

images (34).jpeg


/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ImageScaler in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of IsInf in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of Mod in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ThresholdedRelu in domain `` with max_inclusive_version=9. Set to 1.
  hand

[[195  99 315 305]] [1] [0.9978734]
195 99 315 305
(256, 256, 3)
images (32).jpeg
[[298  71 408 268]] [1] [0.9986474]
298 71 408 268
(256, 256, 3)
images (9).jpeg
[[245 103 417 311]] [1] [0.9999685]
245 103 417 311
(256, 256, 3)
images (21).jpeg
[] [] []
1.jpg
[[243 207 385 396]] [1] [0.99877197]
243 207 385 396
(256, 256, 3)
download (3).jpeg
[[227 127 414 316]] [1] [0.9999994]
227 127 414 316
(256, 256, 3)
images (13).jpeg
[[189 115 441 347]] [1] [0.999645]
189 115 441 347
(256, 256, 3)
images (3).jpeg
[[231 130 414 400]
 [214 165 290 287]] [1 1] [0.9849274  0.72070575]
231 130 414 400
(256, 256, 3)
214 165 290 287
(256, 256, 3)
download (1).jpeg
[[272  51 558 281]] [1] [0.9997067]
272 51 558 281
(256, 256, 3)
images (25).jpeg
[] [] []
images (22).jpeg
[[240  94 474 307]] [1] [0.9999864]
240 94 474 307
(256, 256, 3)
images (2).jpeg
[[211  89 428 241]] [1] [0.79669195]
211 89 428 241
(256, 256, 3)
images (14).jpeg
[[231  68 384 315]] [1] [0.9651954]
231 68 384 315
(256, 256, 3)
images